In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import progressbar
import pickle
import json

In [2]:
# load the original dataset
df = pd.read_csv('data/USvideos.csv')

In [3]:
df.columns

Index(['video_id', 'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'video_error_or_removed', 'description'],
      dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40949 entries, 0 to 40948
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   video_id                40949 non-null  object
 1   trending_date           40949 non-null  object
 2   title                   40949 non-null  object
 3   channel_title           40949 non-null  object
 4   category_id             40949 non-null  int64 
 5   publish_time            40949 non-null  object
 6   tags                    40949 non-null  object
 7   views                   40949 non-null  int64 
 8   likes                   40949 non-null  int64 
 9   dislikes                40949 non-null  int64 
 10  comment_count           40949 non-null  int64 
 11  thumbnail_link          40949 non-null  object
 12  comments_disabled       40949 non-null  bool  
 13  ratings_disabled        40949 non-null  bool  
 14  video_error_or_removed  40949 non-null  bool  
 15  de

In [5]:
# drop rows with missing values
df.dropna(inplace=True)
# df = df.dropna()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40379 entries, 0 to 40948
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   video_id                40379 non-null  object
 1   trending_date           40379 non-null  object
 2   title                   40379 non-null  object
 3   channel_title           40379 non-null  object
 4   category_id             40379 non-null  int64 
 5   publish_time            40379 non-null  object
 6   tags                    40379 non-null  object
 7   views                   40379 non-null  int64 
 8   likes                   40379 non-null  int64 
 9   dislikes                40379 non-null  int64 
 10  comment_count           40379 non-null  int64 
 11  thumbnail_link          40379 non-null  object
 12  comments_disabled       40379 non-null  bool  
 13  ratings_disabled        40379 non-null  bool  
 14  video_error_or_removed  40379 non-null  bool  
 15  de

In [7]:
# drop all unnecessary columns
df.drop(df.columns[[0,1,5,7,8,9,10,11,12,13,14]], axis=1, inplace=True)
# df = df.drop(df.columns[[0,1,5,7,8,9,10,11,12,13,14]], axis=1)

In [8]:
df.drop_duplicates(subset=["title","channel_title","tags","description"], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7184 entries, 0 to 40846
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          7184 non-null   object
 1   channel_title  7184 non-null   object
 2   category_id    7184 non-null   int64 
 3   tags           7184 non-null   object
 4   description    7184 non-null   object
dtypes: int64(1), object(4)
memory usage: 336.8+ KB


# TEST

In [ ]:
# print(df.description[0])
# print(df.tags[1])
# print(df.tags[2])
# print(df.tags[3])
print(df.tags[52])
# print(df.tags[40948])

In [ ]:
df.tags[df.tags == "kpop"].index

In [ ]:
# tokenize and lowercase the strings
tokens = nltk.word_tokenize(str(df.tags[52]))
tokens = [word.lower() for word in tokens]
tokens

In [ ]:
# strip punctuation from words
table = str.maketrans('', '', string.punctuation)
stripped = [w.translate(table) for w in tokens]
stripped

In [ ]:
# remove all non words
words = [word for word in stripped if word.isalpha()]
words

In [ ]:
# remove all non important words
stop_words = nltk.corpus.stopwords.words('english')
words = [w for w in words if not w in stop_words]
words

In [ ]:
# stems words to their roots
porter = nltk.stem.porter.PorterStemmer()
stemmed = [porter.stem(word) for word in words]
stemmed

## END TEST

In [9]:
def clean(df, column):
    '''
        Cleans the strings in specified column
    '''
    list = []
    print(column)
    table = str.maketrans('', '', string.punctuation)
    stop_words = nltk.corpus.stopwords.words('english')
    porter = nltk.stem.porter.PorterStemmer()
#     index = 0
#     import ipdb; ipdb.set_trace()
    for row in progressbar.progressbar(df[column]):
#     for index in progressbar.progressbar(range(0, len(df[column]))):
        index = df[column][df[column] == row].index.tolist()[0]
        tokens = nltk.word_tokenize(row)
#         tokens = nltk.word_tokenize(df[column][index])
        tokens = [word.lower() for word in tokens]
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
        words = [w for w in words if not w in stop_words]
        stemmed = [porter.stem(word) for word in words]
#         list.append(';'.join(stemmed))
        df[column][index] = ';'.join(stemmed)
#         df.iloc[index][column] = ';'.join(stemmed)
#         index += 1
#     return list

In [10]:
%%time
title = clean(df, "title")
channel_title = clean(df, "channel_title")
tags = clean(df, "tags")
description = clean(df, "description")

N/A% (0 of 7184) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--<ipython-input-9-d2545c0232bb>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column][index] = ';'.join(stemmed)
  2% (182 of 7184) |                     | Elapsed Time: 0:00:00 ETA:   0:00:06

title


100% (7184 of 7184) |####################| Elapsed Time: 0:00:06 Time:  0:00:06
  2% (182 of 7184) |                     | Elapsed Time: 0:00:00 ETA:   0:00:05

channel_title


100% (7184 of 7184) |####################| Elapsed Time: 0:00:05 Time:  0:00:05
  1% (91 of 7184) |                      | Elapsed Time: 0:00:00 ETA:   0:00:12

tags


100% (7184 of 7184) |####################| Elapsed Time: 0:00:12 Time:  0:00:12
  0% (49 of 7184) |                      | Elapsed Time: 0:00:00 ETA:   0:00:29

description


100% (7184 of 7184) |####################| Elapsed Time: 0:00:26 Time:  0:00:26


Wall time: 51.1 s


In [ ]:
%%time
table = str.maketrans('', '', string.punctuation)
stop_words = nltk.corpus.stopwords.words('english')
porter = nltk.stem.porter.PorterStemmer()
index = 0
for row in progressbar.progressbar(df.title):
    tokens = nltk.word_tokenize(row)
    tokens = [word.lower() for word in tokens]
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    words = [w for w in words if not w in stop_words]
    stemmed = [porter.stem(word) for word in words]
    df.title[index] = ';'.join(stemmed)
    index += 1
    
index = 0
for row in progressbar.progressbar(df.channel_title):
    tokens = nltk.word_tokenize(row)
    tokens = [word.lower() for word in tokens]
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    words = [w for w in words if not w in stop_words]
    stemmed = [porter.stem(word) for word in words]
    df.description[index] = ';'.join(stemmed)
    index += 1
    
# index = 0
# for row in progressbar.progressbar(df.tags):
#     tokens = nltk.word_tokenize(row)
#     tokens = [word.lower() for word in tokens]
#     stripped = [w.translate(table) for w in tokens]
#     words = [word for word in stripped if word.isalpha()]
#     words = [w for w in words if not w in stop_words]
#     stemmed = [porter.stem(word) for word in words]
#     df.tags[index] = ';'.join(stemmed)
#     index += 1
    
# index = 0
# for row in progressbar.progressbar(df.tags):
#     tokens = nltk.word_tokenize(row)
#     tokens = [word.lower() for word in tokens]
#     stripped = [w.translate(table) for w in tokens]
#     words = [word for word in stripped if word.isalpha()]
#     words = [w for w in words if not w in stop_words]
#     stemmed = [porter.stem(word) for word in words]
#     df.title[index] = ';'.join(stemmed)
#     index += 1

In [ ]:
df.title

In [ ]:
df.channel_title

In [ ]:
df.channel_title[23]

In [ ]:
df.tags

In [ ]:
df.description

In [11]:
# drop all the empty strings
df.replace('', float("NaN"), inplace=True)
df.dropna(inplace=True)

In [12]:
df

,title,channel_title,category_id,tags,description
0,want;talk;marriag,caseyneistat,22,shantel;martin,shantel;channel;http;wwwyoutubecomshantellmart...
1,trump;presid;last;week;tonight;john;oliv;hbo,lastweektonight,24,last;week;tonight;trump;presid;last;week;tonig...,one;year;presidenti;elect;john;oliv;discuss;le...
2,racist;superman;rudi;mancuso;king;bach;lele;pon,rudi;mancuso,23,racist;superman;rudi;mancuso;king;bach;racist;...,watch;previou;video;nnsubscrib;http;watch;like...
3,nickelback;lyric;real;fake,good;mythic;morn,24,rhett;link;gmm;good;mythic;morn;rhett;link;goo...,today;find;link;nickelback;amateur;secret;nick...
4,dare;go;bald,nigahiga,24,ryan;higa;higatv;nigahiga;dare;idi;rhpc;dare;t...,know;sinc;show;back;might;best;episod;yet;nlea...
...,...,...,...,...,...
40762,drop;mic;w;ashton;kutcher;sean;diddi;comb,late;late;show;jame;corden,24,late;late;show;late;late;show;jame;corden;cord...,jame;corden;ashton;kutcher;squar;rap;battl;tak...
40764,carla;make;ba;smashburg;test;kitchen;bon;appétit,bon;appétit,26,bon;appetit;burger;cheeseburg;cook;test;kitche...,ground;chuck;great;allpurpos;buyitanywher;choi...
40766,katherin;langford;reason;australia;doctor;parent,jimmi;kimmel;live,24,jimmi;kimmel;live;late;night;talk;show;funni;c...,katherin;talk;learn;accent;grow;australia;pare...
40793,battlefield;offici;multiplay;trailer,battlefield,20,battlefield;battlefield;trailer;bfv;battlefiel...,v;launch;worldwid;octob;preorder;open;beta;ear...


In [13]:
with open('data/US_category_id.json', 'r') as f:
    data = json.load(f)
    for item in data['items']:
#         df.category_id.replace(item['snippet']['title'], item['id'], inplace=True)
        df.replace({'category_id': int(item['id'])}, item['snippet']['title'], inplace=True)

In [14]:
df.category_id

0        People & Blogs
1         Entertainment
2                Comedy
3         Entertainment
4         Entertainment
              ...      
40762     Entertainment
40764     Howto & Style
40766     Entertainment
40793            Gaming
40846             Music
Name: category_id, Length: 6997, dtype: object

In [15]:
# drop all the nones
df.replace('none', float("NaN"), inplace=True)
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6757 entries, 0 to 40846
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          6757 non-null   object
 1   channel_title  6757 non-null   object
 2   category_id    6757 non-null   object
 3   tags           6757 non-null   object
 4   description    6757 non-null   object
dtypes: object(5)
memory usage: 316.7+ KB


## Save Dataframe

In [16]:
df.to_csv('data/clean.csv', columns=['title','channel_title','category_id','tags','description'], index=False)

In [ ]:
%%time
# save df object
with open('data/df.pkl', 'wb') as f:
    pickle.dump(df, f)

## Load Dataframe

In [ ]:
%%time
# save df object
with open('data/df.pkl', 'rb') as f:
    df = pickle.load(f)

In [ ]:
df.info()

In [ ]:
df.tags[52]